In [40]:
import random

def generate_dummy_dataset(filename, num_elements):
    with open(filename, 'w') as file:
        for _ in range(num_elements):
            file.write(str(random.randint(1, 1000)) + '\n')

# Generate a file 'dummy_dataset.txt' with 1000 random integers
generate_dummy_dataset('dummy_dataset.txt', 1000)

In [41]:
import heapq
import sys
import os

def read_chunk(file, chunk_size):
    chunk = []
    for _ in range(chunk_size):
        line = file.readline()
        if not line:
            break
        chunk.append(int(line.strip()))
    # print(f"{chunk = }")
    return chunk

In [42]:
def external_merge_sort(input_file, output_file, chunk_size):
    # Phase 1: Sort individual chunks internally
    with open(input_file, 'r') as infile:
        chunk_count = 0
        while True:
            chunk = read_chunk(infile, chunk_size)
            if not chunk:
                break
            chunk.sort()
            chunk_count += 1
            with open(f'chunk_{chunk_count}.txt', 'w') as chunk_file:
                for item in chunk:
                    # print(f"{item =}")
                    chunk_file.write(str(item) + '\n')
                    
    chunks = []
    # Phase 2: Merge sorted chunks
    for i in range(1, chunk_count+1):
        chunks.append(open(f'chunk_{i}.txt'))
        
    heap = []    
    with open(output_file, 'w') as outfile:
        for i, chunk in enumerate(chunks):
            heap.append((int(chunk.readline().strip()), i))
            heapq.heapify(heap)
        print(f"min-heap: {heap}")
        print(".......................................................")
        while heap:
            print("########################################################")
            smallest, chunk_idx = heapq.heappop(heap)
            print(f"{smallest = }, {chunk_idx =}")
            print(f"heappop: {heap}")
            
            outfile.write(f'{smallest}\n')
            next_item = chunks[chunk_idx].readline().strip()
            if next_item:
                heapq.heappush(heap, (int(next_item), chunk_idx))
                print(f"heappush: {heap}")

    # Clean up temporary chunk files
    for i in range(1, chunk_count+1):
        filename = f'chunk_{i}.txt'
        os.remove(filename)

# Perform external merge sort on 'dummy_dataset.txt'
external_merge_sort('dummy_dataset.txt', 'sorted_dataset.txt', chunk_size=100)

min-heap: [(2, 3), (3, 4), (6, 2), (8, 0), (5, 1), (28, 5), (11, 6), (22, 7), (24, 8), (5, 9)]
.......................................................
########################################################
smallest = 2, chunk_idx =3
heappop: [(3, 4), (5, 1), (6, 2), (8, 0), (5, 9), (28, 5), (11, 6), (22, 7), (24, 8)]
heappush: [(3, 4), (5, 1), (6, 2), (8, 0), (5, 9), (28, 5), (11, 6), (22, 7), (24, 8), (7, 3)]
########################################################
smallest = 3, chunk_idx =4
heappop: [(5, 1), (5, 9), (6, 2), (8, 0), (7, 3), (28, 5), (11, 6), (22, 7), (24, 8)]
heappush: [(5, 1), (5, 9), (6, 2), (8, 0), (7, 3), (28, 5), (11, 6), (22, 7), (24, 8), (8, 4)]
########################################################
smallest = 5, chunk_idx =1
heappop: [(5, 9), (7, 3), (6, 2), (8, 0), (8, 4), (28, 5), (11, 6), (22, 7), (24, 8)]
heappush: [(5, 9), (7, 3), (6, 2), (8, 0), (8, 4), (28, 5), (11, 6), (22, 7), (24, 8), (31, 1)]
#####################################################

In [43]:
import heapq

# Initial list
lst = [4, 3, 7, 1, 9, 2]

# Turn the list into a min-heap
heapq.heapify(lst)

print(lst)

[1, 3, 2, 4, 9, 7]


In [27]:
import heapq

# Create a min-heap
heap = [3, 1, 4, 1, 5, 9, 2]

# Retrieve and remove the first element
smallest = heapq.heappop(heap)

print(f"The first element is: {smallest}")
print(f"The updated heap is: {heap}")

The smallest element is: 3
The updated heap is: [1, 1, 4, 2, 5, 9]


-----

In [48]:
import random

def generate_dummy_dataset(filename):
    country_name = ['country_name', 'USA', 'Canada', 'Germany', 'France', 'UK']
    year = ['year', 2020, 2019, 2018, 2021, 2017]
    with open(filename, 'w') as file:
        for idx, ele in enumerate(country_name):
            file.write(str(country_name[idx]) + ',' + str(year[idx]) + '\n')

generate_dummy_dataset('dummy_data.csv')

----

In [42]:
import csv
import shutil
import random
import tempfile
import pandas as pd

In [45]:
# Generate a dummy dataset with 100 rows
data = []

for _ in range(100):
    country = random.choice(["CountryA", "CountryB", "CountryC"])
    year = random.randint(1900, 2023)
    data.append([country, year])

# Shuffle the data to make it unordered
random.shuffle(data)

# Write the data to a CSV file
with open('dummy_data.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    # writer.writerow(['country_name', 'year'])
    writer.writerows(data)

In [46]:
def sort_chunks(chunk_paths, output_path):
    # Read chunks, sort, and write to temporary files
    sorted_chunk_paths = []
    for chunk_path in chunk_paths:
        with open(chunk_path, 'r') as chunk_file:
            chunk = list(csv.reader(chunk_file))
            chunk.sort(key=lambda x: (x[0], int(x[1])))

            sorted_chunk_path = tempfile.mktemp()
            with open(sorted_chunk_path, 'w', newline='') as sorted_chunk_file:
                writer = csv.writer(sorted_chunk_file)
                writer.writerows(chunk)

            sorted_chunk_paths.append(sorted_chunk_path)

    # Merge sorted chunks
    with open(output_path, 'w', newline='') as output_file:
        writer = csv.writer(output_file)
        readers = [csv.reader(open(sorted_chunk_path, 'r')) for sorted_chunk_path in sorted_chunk_paths]
        merged = heapq.merge(*readers, key=lambda x: (x[0], int(x[1])))
        writer.writerows(merged)

    # Clean up temporary files
    for sorted_chunk_path in sorted_chunk_paths:
        os.remove(sorted_chunk_path)

In [49]:
input_path = 'dummy_data.csv'
output_path = 'sorted_data.csv' 
chunk_size=9 # chunk_size=10, but the chunk list stores 11 rows, so I ma gonna make chunk_size=9.

# Step 1: Read the input data and divide into chunks
chunk_paths = []
with open(input_path, 'r') as input_file:
    reader = csv.reader(input_file)
    header = next(reader)
    # print(f"{header = }")
    chunk = [header]
    count = 0
    
    print("############ 1 ############ ")
    for row in reader:
        # print(f"{row = }")
        chunk.append(row)
        print(f"{chunk = }")
        count += 1
        '''
        chunk_size=10, but the chunk list stores 11 rows, so I am gonna make chunk_size=9.
        '''
        if count == chunk_size:
            chunk_path = tempfile.mktemp()
            print(f"{chunk_path = }")
            with open(chunk_path, 'w', newline='') as chunk_file:
                writer = csv.writer(chunk_file)
                writer.writerows(chunk)
            chunk_paths.append(chunk_path)
            print("############ 2 ############ ")
            # print(f"{header = }")
            # chunk = [header]
            chunk = []
            print(f"{chunk = }")
            count = 0
    print("########################## ")
        
# Step 2: Sort and merge chunks
sort_chunks(chunk_paths, output_path)

# Clean up temporary chunk files
for chunk_path in chunk_paths:
    os.remove(chunk_path)

############ 1 ############ 
chunk = [['CountryA', '1948'], ['CountryC', '1975']]
chunk = [['CountryA', '1948'], ['CountryC', '1975'], ['CountryC', '1972']]
chunk = [['CountryA', '1948'], ['CountryC', '1975'], ['CountryC', '1972'], ['CountryB', '2001']]
chunk = [['CountryA', '1948'], ['CountryC', '1975'], ['CountryC', '1972'], ['CountryB', '2001'], ['CountryB', '1907']]
chunk = [['CountryA', '1948'], ['CountryC', '1975'], ['CountryC', '1972'], ['CountryB', '2001'], ['CountryB', '1907'], ['CountryC', '1909']]
chunk = [['CountryA', '1948'], ['CountryC', '1975'], ['CountryC', '1972'], ['CountryB', '2001'], ['CountryB', '1907'], ['CountryC', '1909'], ['CountryA', '1990']]
chunk = [['CountryA', '1948'], ['CountryC', '1975'], ['CountryC', '1972'], ['CountryB', '2001'], ['CountryB', '1907'], ['CountryC', '1909'], ['CountryA', '1990'], ['CountryA', '1915']]
chunk = [['CountryA', '1948'], ['CountryC', '1975'], ['CountryC', '1972'], ['CountryB', '2001'], ['CountryB', '1907'], ['CountryC', '1909'

In [50]:
# Load sorted data into a pandas dataframe
sorted_df = pd.read_csv('sorted_data.csv')
print(sorted_df)

    CountryA  1905
0   CountryA  1915
1   CountryA  1920
2   CountryA  1920
3   CountryA  1922
4   CountryA  1929
..       ...   ...
94  CountryC  2003
95  CountryC  2004
96  CountryC  2018
97  CountryC  2020
98  CountryC  2022

[99 rows x 2 columns]


----

## Try a much more realistic dataset like ours

In [ ]:
import csv
import shutil
import random
import tempfile
import pandas as pd

def sort_chunks(chunk_paths, output_path):
    # Read chunks, sort, and write to temporary files
    sorted_chunk_paths = []
    for chunk_path in chunk_paths:
        with open(chunk_path, 'r') as chunk_file:
            chunk = list(csv.reader(chunk_file))
            chunk.sort(key=lambda x: (x[0], int(x[1])))

            sorted_chunk_path = tempfile.mktemp()
            with open(sorted_chunk_path, 'w', newline='') as sorted_chunk_file:
                writer = csv.writer(sorted_chunk_file)
                writer.writerows(chunk)

            sorted_chunk_paths.append(sorted_chunk_path)

    # Merge sorted chunks
    with open(output_path, 'w', newline='') as output_file:
        writer = csv.writer(output_file)
        readers = [csv.reader(open(sorted_chunk_path, 'r')) for sorted_chunk_path in sorted_chunk_paths]
        merged = heapq.merge(*readers, key=lambda x: (x[0], int(x[1])))
        writer.writerows(merged)

    # Clean up temporary files
    for sorted_chunk_path in sorted_chunk_paths:
        os.remove(sorted_chunk_path)

In [52]:
input_path = 'sample_preprocessed.csv'
output_path = 'sorted_sample_preprocessed.csv' 
chunk_size=9 # chunk_size=10, but the chunk list stores 11 rows, so I ma gonna make chunk_size=9.

# Step 1: Read the input data and divide into chunks
chunk_paths = []
with open(input_path, 'r') as input_file:
    reader = csv.reader(input_file)
    header = next(reader)
    # print(f"{header = }")
    chunk = [header]
    count = 0
    
    print("############ 1 ############ ")
    for row in reader:
        # print(f"{row = }")
        chunk.append(row)
        print(f"{chunk = }")
        count += 1
        '''
        chunk_size=10, but the chunk list stores 11 rows, so I am gonna make chunk_size=9.
        '''
        if count == chunk_size:
            chunk_path = tempfile.mktemp()
            print(f"{chunk_path = }")
            with open(chunk_path, 'w', newline='') as chunk_file:
                writer = csv.writer(chunk_file)
                writer.writerows(chunk)
            chunk_paths.append(chunk_path)
            print("############ 2 ############ ")
            # print(f"{header = }")
            # chunk = [header]
            chunk = []
            print(f"{chunk = }")
            count = 0
    print("########################## ")
        
# Step 2: Sort and merge chunks
sort_chunks(chunk_paths, output_path)

# Clean up temporary chunk files
for chunk_path in chunk_paths:
    os.remove(chunk_path)

############ 1 ############ 
chunk = [['Albania', '1987', 'male', '15-24 years', 'Generation X', 'low_suicides_range', 'medium_population_range', 'low_income_range', '0-2000'], ['Albania', '2004', 'female', '75+ years', 'Silent', 'low_suicides_range', 'low_population_range', 'low_income_range', '2000-4000']]
chunk = [['Albania', '1987', 'male', '15-24 years', 'Generation X', 'low_suicides_range', 'medium_population_range', 'low_income_range', '0-2000'], ['Albania', '2004', 'female', '75+ years', 'Silent', 'low_suicides_range', 'low_population_range', 'low_income_range', '2000-4000'], ['Albania', '2004', 'female', '25-34 years', 'Generation X', 'low_suicides_range', 'medium_population_range', 'low_income_range', '2000-4000']]
chunk = [['Albania', '1987', 'male', '15-24 years', 'Generation X', 'low_suicides_range', 'medium_population_range', 'low_income_range', '0-2000'], ['Albania', '2004', 'female', '75+ years', 'Silent', 'low_suicides_range', 'low_population_range', 'low_income_range'